In [ ]:
#vamos a probar modelos de ML
#leamos x_train, y_train, x_test, y_test
import pandas as pd
x_train = pd.read_csv("x_train.csv")
y_train = pd.read_csv("y_train.csv")
x_test = pd.read_csv("x_val.csv")
y_test = pd.read_csv("y_val.csv")




In [9]:
# En este modelo
# i) no estoy haciendo busqueda de hiperparámetros
# iii) no estoy seleccionando variables (probablmente podría eliminar 90 de las 120 features que estoy usando)
# iv) solo estoy usando 1 modelo (Random Forest)
# El modelo debe funcionar bien porque definí algunas características importantes en el dataset (lags, rolling means, etc)
# pero no es un modelo optimizado ni final. Solo es para tener una línea base de desempeño.
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson')
X_train_ready = pd.DataFrame(pt.fit_transform(x_train), columns=x_train.columns)
X_test_ready = pd.DataFrame(pt.transform(x_test), columns=x_test.columns)
model = RandomForestRegressor(n_estimators=1000, random_state=42)
model.fit(X_train_ready, y_train)
y_pred = model.predict(X_test_ready)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse:.2f}")
print(f"R2: {r2:.2f}")


c:\Users\agusm\Trabajo-Final-Lab-Datos-\.venv\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


MSE: 11366494.07
R2: 0.28


In [10]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.2f}")

MAE: 2640.42
